In [12]:
# # uncomment this cell to run on kaggle server
# import os
# os.listdir('../input/rl-project')
# import sys
# sys.path.insert(0,'../input/rl-project/')

In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from tqdm import tqdm

from functools import partial

from IPython.display import clear_output

from torch.utils.tensorboard import SummaryWriter
import socket
from datetime import datetime
import os

from agents import Agent
from environment import SimulationEnvironment0
from replay_buffers import *
from utils import *

import copy


In [14]:
torch.autograd.set_detect_anomaly(False)

In [16]:
seed = 0

EXPERIMENTS = [ #{'PPO':2, 'replay_ratio':0, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True,'input_type':'complete', 'name':'PPO_SAC2', 'run':'ppo2' },
                {'PPO':2, 'replay_ratio':1, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True,'input_type':'complete', 'name':'PPO_SAC2', 'run':'ppo4' },
                #{'PPO':8, 'replay_ratio':0, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True,'input_type':'complete', 'name':'PPO_SAC2', 'run':'ppo8' },

                # {'PPO':2, 'replay_ratio':0, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'PPO_SAC', 'run':'ppo2' },
                # {'PPO':4, 'replay_ratio':0, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'PPO_SAC', 'run':'ppo4' },
                # {'PPO':8, 'replay_ratio':0, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'PPO_SAC', 'run':'ppo8' },

                # {'PPO':2, 'replay_ratio':1, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'ppo_sac_acer', 'run':'ppo2_rr1' },
                # {'PPO':2, 'replay_ratio':4, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'ppo_sac_acer', 'run':'ppo2_rr4' },
                # {'PPO':2, 'replay_ratio':8, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'ppo_sac_acer', 'run':'ppo2_rr8' },

                # {'PPO':4, 'replay_ratio':1, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'ppo_sac_acer', 'run':'ppo4_rr1' },
                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'ppo_sac_acer', 'run':'ppo4_rr4' },
                # {'PPO':4, 'replay_ratio':8, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'ppo_sac_acer', 'run':'ppo4_rr8' },

                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':False,'use_symlog':False,'use_SAC':True, 'input_type':'complete', 'name':'continuous', 'run':'NOsym_sac' },
                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':False,'use_symlog':True, 'use_SAC':False,'input_type':'complete', 'name':'continuous', 'run':'sym_NOsac' },
                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':False,'use_symlog':False,'use_SAC':False,'input_type':'complete', 'name':'continuous', 'run':'NOsym_NOsac' },

                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'augmented','name':'network', 'run':'augmented',},
                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':False,'use_symlog':True, 'use_SAC':True, 'input_type':'base',     'name':'network', 'run':'base',     },
                
                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':True, 'use_symlog':True, 'use_SAC':True, 'input_type':'complete', 'name':'discrete', 'run':'sym_sac' },
                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':True, 'use_symlog':False,'use_SAC':True, 'input_type':'complete', 'name':'discrete', 'run':'NOsym_sac' },
                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':True, 'use_symlog':True, 'use_SAC':False,'input_type':'complete', 'name':'discrete', 'run':'sym_NOsac' },
                # {'PPO':4, 'replay_ratio':4, 'discrete_actions':True, 'use_symlog':False,'use_SAC':False,'input_type':'complete', 'name':'discrete', 'run':'NOsym_NOsac' },
               ]


# simulation
num_simulations = 128
num_blackholes = 1

# agent
hidden_size = 512
simlog_res = 255
#use_symlog = True
#discrete_actions = True
simlog_half_res = simlog_res//2
simlog_max_range = 1
actions_res = 5
levels=2
input_type = 'complete'

lr = 3e-4
lr_actor = 3e-5

#use_SAC = False


# training
training_steps=2**17
lamb = 0.8
gamma = 0.98
smoothing = 1e-2
eps = 0.05 # for PPO update
seg_len = 2**5
c=1
entropy = 3e-4


# replay buffers
num_steps = 1024
batch_size = 2**10
replay_batch_size = num_simulations

plot = False

validate_every = 2**9

bin_values = (torch.arange(simlog_res)-simlog_half_res).cuda()/simlog_half_res*simlog_max_range
bin_values = bin_values.sign()*(bin_values.abs().exp()-1)

dec_x, dec_y = torch.meshgrid(torch.arange(actions_res)/(actions_res-1)*2-1, torch.arange(actions_res)/(actions_res-1)*2-1)
dec_x, dec_y = dec_x.flatten().cuda(), dec_y.flatten().cuda()

metric_idx = torch.pow(2,torch.arange(15))-1

fig, ax = plt.subplots(figsize=(10,10))

for experiment in EXPERIMENTS:

    # SETUP EXPERIMENT

    # set seed
    torch.manual_seed(seed)

    # initialize logging (with tensorboard)
    current_time = datetime.now().strftime("%b%d_%H-%M-%S")
    log_dir = os.path.join(
        "runs",experiment['name'], experiment['run'], str(seed), current_time + "_" + socket.gethostname() 
    )
    tb_writer = SummaryWriter(log_dir)

    # initialize the simulation
    sim = SimulationEnvironment0(num_simulations=num_simulations,
                            num_blackholes=num_blackholes, 
                            force_constant=0.002, 
                            velocity_scale=0.01,
                            goal_threshold=0.05,
                            max_steps=250,
                            device='cuda')
    states = sim.get_state()

    # initialize the networks
    if experiment['discrete_actions']:
        action_dim = actions_res**2             # output shape of the actor
        compute_dist = compute_dist_discrete
        decode_action = partial(decode_action_discrete, dec_x=dec_x, dec_y=dec_y)
        action_shape = ()                     # shape after sampling the dist, before decoding
    else:
        action_dim = 5
        compute_dist = compute_dist_continuous
        decode_action = decode_action_continuous
        action_shape = (2,)

    if experiment['use_symlog']:
        value_dimension = simlog_res
        decode_values = partial(decode_value_symlog, bin_values)
        critic_error_func = partial(critic_error_func_symlog, simlog_max_range, simlog_res, simlog_half_res, smoothing)
    else:
        value_dimension = 1
        decode_values = lambda x : x[:,0]
        critic_error_func = critic_error_func_normal

    actor = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, critic=False, action_dimension=action_dim).cuda()
    actor2 = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, critic=False, action_dimension=action_dim).cuda()
    V = Agent((num_blackholes+2)*2, hidden_size, levels, input_type, actor=False, value_dimension=value_dimension).cuda()
    Q = Agent(hidden_size + 2, hidden_size, levels, input_type='base', actor=False, value_dimension=value_dimension).cuda()


    optim_actor = torch.optim.AdamW(list(actor.parameters())+list(actor2.parameters()), lr=lr_actor, weight_decay=1e-3)
    optim_critic = torch.optim.AdamW(list(V.parameters())+list(Q.parameters()), lr=lr, weight_decay=1e-3)
    target_actor = copy.deepcopy(actor)
    target_V = copy.deepcopy(V)
    target_Q = copy.deepcopy(Q)

    update_models = partial(update_target_models, actor, V, Q, target_actor, target_V, target_Q)

    # initialize the replay buffer
    replay_buffer = Replay_Buffer_Segments(state_shape=((num_blackholes+2),2), 
                                            action_shape=action_shape, 
                                            params_shape=(action_dim,), 
                                            segment_lenght=seg_len, 
                                            num_simulations=num_simulations, 
                                            num_steps=num_steps, 
                                            batch_size=replay_batch_size, 
                                            device='cuda')

    # initialize vailidation plane
    x,y = torch.meshgrid(torch.arange(100),torch.arange(100))
    pos = torch.stack([x.flatten(), y.flatten()],1)/100

    target_pos = torch.ones_like(pos)*0.25      # position of the target
    bh_pos = torch.ones_like(pos)*0.75          # position of the blackholes

    st=torch.stack([pos,target_pos,bh_pos],1)

    plots = partial(validation_plots, tb_writer, st, batch_size, V, target_V, actor, decode_values, experiment['discrete_actions'], dec_x, dec_y, fig, ax)


    # START EXPERIMENT
    pbar = tqdm(range(training_steps))
    for i in pbar:
        t0=time.time()

        # GENERATE EXPERIENCE
        with torch.inference_mode():
            
            states = states.reshape(states.shape[0],-1).cuda()

            # compute action distribution according to the current policy
            actions, _, _ = actor(states)
            dist = compute_dist(actions)

            # sample an action
            sampled_action = dist.sample()
            sampled_action_decoded, log_prob = decode_action(dist, sampled_action)

            # simulation step
            rewards, new_states, terminals = sim.step(sampled_action_decoded)

            # save experience
            replay_buffer.add_experience(states.reshape(new_states.shape), sampled_action, actions, log_prob, rewards, terminals)

        tb_writer.add_scalar('Reward',rewards.mean().item(), i)

        if (i+1) % seg_len == 0:

            # ON POLICY UPDATE

            seg_state, seg_actions, _, seg_action_log_probs, seg_rewards, seg_terminal = replay_buffer.get_last_segment()

            with torch.inference_mode():

                # get target values for all the states in the segment
                V_t = torch.zeros(num_simulations*seg_len, device='cuda')
                for b_idx in range((seg_len*num_simulations+batch_size-1)//batch_size):

                    b_state = seg_state.reshape((seg_len)*num_simulations,-1)[b_idx*batch_size:(b_idx+1)*batch_size]
                    
                    _, Vs, _= target_V(b_state)
                    V_t[b_idx*batch_size:(b_idx+1)*batch_size] = decode_values(Vs)
        
                V_t = V_t.reshape(num_simulations, seg_len)

                # compute GAE and TD lambda returns
                gae = torch.zeros_like(V_t)
                for t in reversed(range(seg_len-1)):

                    d_t = -V_t[:,t] + seg_rewards[:,t] + gamma*V_t[:,t+1]*seg_terminal[:,t].logical_not()

                    if experiment['use_SAC']:
                        d_t = d_t - seg_action_log_probs[:,t].exp()*entropy

                    gae[:,t] = d_t + gamma*lamb*gae[:,t+1]*seg_terminal[:,t].logical_not()

                tdl = V_t + gae

            # unite batch and sequence dimensions
            [gae, tdl, seg_state, seg_actions, seg_action_log_probs] = flatten_sequences([gae, tdl, seg_state, seg_actions, seg_action_log_probs], removelast=True)

            # run PPO for n epochs
            for ppo_epoch in range(experiment['PPO']):
                for b_idx in range((seg_len*num_simulations+batch_size-1)//batch_size):

                    # get batch
                    [b_gae, b_tdl, b_state, b_action, b_log_prob] = get_batch([gae, tdl, seg_state, seg_actions, seg_action_log_probs], batch_size, b_idx)

                    # compute action distribution according to the updated policy
                    actions, _, _ = actor(b_state)
                    dist = compute_dist(actions)

                    # compute probability of the chosen action with respect to the updated policy
                    sampled_action_decoded, log_prob = decode_action(dist, b_action)

                    # compute the value functions
                    _, Vs, hs = V(b_state)
                    _, Qs, _ = Q(torch.concat([hs, sampled_action_decoded],-1))

                    # compute PPO clip objective
                    r = (log_prob.exp()+1e-5) / (b_log_prob.exp()+1e-5)
                    L = torch.minimum(b_gae*r, b_gae*r.clip(1-eps,1+eps))

                    # approximate entropy
                    H = - log_prob

                    actor_error = - L + 1e-5*actions.square().mean()
                    if not experiment['use_SAC']:
                        actor_error = actor_error + log_prob.exp()*entropy*(1-i/training_steps)

                    critic_error_V = critic_error_func(Vs, b_tdl) 
                    critic_error_Q = critic_error_func(Vs+Qs, b_tdl)
                    critic_error = (critic_error_V + critic_error_Q) / 2
                    error = actor_error + critic_error

                    optim_critic.zero_grad()
                    optim_actor.zero_grad() 
                    error.mean().backward()
                    optim_critic.step()
                    optim_actor.step()
                
                update_models(decay=5e-3)

            tb_writer.add_scalar('critic_error_V',critic_error_V.mean().item(), i)
            tb_writer.add_scalar('critic_error_Q',critic_error_Q.mean().item()-critic_error_V.mean().item(), i)
            tb_writer.add_scalar('hentropy',H.mean().item(), i)

            # REPLAY EXPERIENCES

            # replay experiences
            for replay_epoch in range(experiment['replay_ratio']):

                seg_state, seg_actions, seg_action_params, seg_action_log_probs, seg_rewards, seg_terminal = replay_buffer.get_batch()

                with torch.inference_mode():

                    # get target values and action probs for all the states in the segment

                    V_t, Q_t, log_probs_t, Q_t_corr, log_probs_corr, log_probs_seg_corr = initialize_zeros(replay_batch_size*seg_len, n=6, device='cuda')
                    action_corr = torch.zeros((replay_batch_size*seg_len,) + action_shape, device='cuda')

                    for b_idx in range((seg_len*replay_batch_size+batch_size-1)//batch_size):

                        # get batch
                        [b_state, b_action, b_seg_action_params] = get_batch(flatten_sequences([seg_state, seg_actions, seg_action_params]), batch_size, b_idx)

                        # compute action distribution according to current policy
                        actions, _, _ = target_actor(b_state)
                        dist = compute_dist(actions)

                        # compute the probability of the replayed action
                        sampled_action_decoded, log_prob = decode_action(dist, b_action)

                        # sample a new action from the current policy (for the bias correction)
                        sampled_action_corr = dist.sample()
                        sampled_action_decoded_corr, log_prob_corr = decode_action(dist, sampled_action_corr)

                        # compute the probabilty of the action sampled with the current policy with respect to the old policy (for the ro in bias correction)
                        seg_dist = compute_dist(b_seg_action_params)
                        _, seg_log_prob_corr = decode_action(seg_dist, sampled_action_corr)

                        # compute the value functions
                        _, Vs, hs = target_V(b_state)
                        _, Qs, _ = target_Q(torch.concat([hs, sampled_action_decoded],-1))
                        _, Qs_corr, _ = target_Q(torch.concat([hs, sampled_action_decoded_corr],-1))

                        b_V_t = decode_values(Vs)
                        b_Q_t = decode_values(Vs+Qs)
                        b_Q_t_corr = decode_values(Vs+Qs_corr)

                        update_batched( [V_t, Q_t, log_probs_t, Q_t_corr, action_corr, log_probs_corr, log_probs_seg_corr],
                                        [b_V_t, b_Q_t, log_prob, b_Q_t_corr, sampled_action_corr, log_prob_corr, seg_log_prob_corr],
                                        batch_size, b_idx)

                    # reshape to expose the sequences
                    [V_t, Q_t, log_probs_t, Q_t_corr, action_corr, log_probs_corr, log_probs_seg_corr] = reshape_sequences([V_t, Q_t, log_probs_t, Q_t_corr, action_corr, log_probs_corr, log_probs_seg_corr], (replay_batch_size, seg_len))


                    # compute targets (as in RETRACE)
                    Q_ret = torch.zeros_like(V_t)
                    V_target = torch.zeros_like(V_t)

                    Q_ret[:,-1] = Q_t[:,-1]

                    for t in reversed(range(seg_len-1)):

                        ro = (log_probs_t[:,t+1].exp()+1e-5)/(seg_action_log_probs[:,t+1].exp()+1e-5) # ro of t+1
                        ci = lamb * torch.minimum(torch.ones(1, device='cuda'), ro)

                        Q_ret[:,t] = seg_rewards[:,t] + gamma*(ci*(Q_ret[:,t+1] - Q_t[:,t+1]) + V_t[:,t+1])

                        if experiment['use_SAC']:
                            Q_ret[:,t] = Q_ret[:,t] - log_probs_t[:,t].exp()*entropy*(1-i/training_steps)

                        V_target[:, t+1] = ci*(Q_ret[:,t+1] - Q_t[:,t+1]) + V_t[:,t+1]

                    V_target[:, 0] = ci*(Q_ret[:,0] - Q_t[:,0]) + Q_t_corr[:,0]

                # unite batch and sequence dimensions
                [Q_ret, V_target, seg_state, seg_actions, log_probs_t, seg_action_log_probs] = flatten_sequences([Q_ret, V_target, seg_state, seg_actions, log_probs_t, seg_action_log_probs], removelast=True)
                [Q_t_corr, V_t, action_corr, log_probs_corr, log_probs_seg_corr] = flatten_sequences([Q_t_corr, V_t, action_corr, log_probs_corr, log_probs_seg_corr], removelast=True)

                # run PPO for n epochs
                for ppo_epoch in range(experiment['PPO']):
                    for b_idx in range((seg_len*replay_batch_size+batch_size-1)//batch_size):

                        # get batch
                        [b_Q_ret, b_V_target, b_state, b_action, b_log_prob, b_log_prob_seg] = get_batch([Q_ret, V_target, seg_state, seg_actions, log_probs_t, seg_action_log_probs], batch_size, b_idx)
                        [b_Q_t_corr, b_V_t, b_action_corr, b_log_probs_corr, b_log_probs_seg_corr] = get_batch([Q_t_corr, V_t, action_corr, log_probs_corr, log_probs_seg_corr], batch_size, b_idx) # bias correction

                        # compute action distribution according to current policy
                        actions, _, _ = actor(b_state)
                        dist = compute_dist(actions)

                        # compute probability of the chosen action with respect to the updated policy
                        sampled_action_decoded, log_prob = decode_action(dist, b_action)

                        # compute probability of the bias-correction action with respect to the updated policy
                        _, log_prob_corr = decode_action(dist, b_action_corr)

                        # compute the value functions
                        _, Vs, hs = V(b_state)
                        _, Qs, _ = Q(torch.concat([hs, sampled_action_decoded],-1))

                        # compute the PPO objective (with respect to the target policy) and apply off policy correction (with bias correction)
                        r = (log_prob.exp()+1e-5)/(b_log_prob.exp()+1e-5)
                        adv = ((b_log_prob.exp()+1e-5)/(b_log_prob_seg.exp()+1e-5)).clip(max = c)*(b_Q_ret - b_V_target)

                        r_c = (log_prob_corr.exp()+1e-5)/(b_log_probs_corr.exp()+1e-5)
                        adv_c = (1 - c*(b_log_probs_seg_corr.exp()+1e-5)/(b_log_probs_corr.exp()+1e-5)).relu()*(b_Q_t_corr - b_V_t)

                        L = torch.minimum(adv*r, adv*r.clip(1-eps,1+eps)) + torch.minimum(adv_c*r_c, adv_c*r_c.clip(1-eps,1+eps))

                        # approximate entropy
                        H = - log_prob

                        actor_error = - L + 1e-5*actions.square().mean()
                        if not experiment['use_SAC']:
                            actor_error = actor_error + log_prob.exp()*entropy

                        critic_error_V = critic_error_func(Vs, b_V_target)
                        critic_error_Q = critic_error_func(Vs+Qs, b_Q_ret )
                        critic_error = (critic_error_V + critic_error_Q) / 2
                        error = actor_error + critic_error

                        optim_critic.zero_grad()
                        optim_actor.zero_grad() 
                        error.mean().backward()
                        optim_critic.step()
                        optim_actor.step()

                    update_models(decay=5e-3)


        states = new_states

        if (i+1) % validate_every == 0:
            plots(i)

100%|██████████| 131072/131072 [33:45<00:00, 64.72it/s]


<Figure size 1000x1000 with 0 Axes>